In [19]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, concatenate, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adamax

In [20]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train / 255., x_test / 255.
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

In [21]:
model_nas = VGG19(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
model_nas.trainable = False

In [22]:
model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) # no pooling
model_vgg.trainable = True

In [23]:
model_nas_out = model_nas.output
model_nas_out = GlobalAveragePooling2D()(model_nas_out)
model_vgg_out = model_vgg.output
model_vgg_out = GlobalAveragePooling2D()(model_vgg_out)
x = concatenate([model_nas_out, model_vgg_out])
x = Dense(10, activation='softmax')(x)

In [24]:
model = Model(inputs=[model_nas.input, model_vgg.input], outputs=x)
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

ValueError: The name "block1_conv1" is used 2 times in the model. All layer names should be unique.

In [7]:
res = model.fit([x_train, x_train], y_train, batch_size=50, epochs=10, validation_data=([x_test, x_test], y_test)) #100

Epoch 1/10


2023-08-01 09:10:12.241313: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8ccc020870 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-08-01 09:10:12.241757: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-08-01 09:10:12.401448: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 18/167 [==>...........................] - ETA: 7:19 - loss: 0.4142 - accuracy: 0.1035

KeyboardInterrupt: 

In [ ]:
htr = res.history
loss = htr['loss']
val_loss = htr['val_loss']
accuracy = htr['accuracy']
val_accuracy = htr['val_accuracy']
epochs = range(1, len(accuracy) + 1)

In [ ]:
plt.plot(epochs, loss, label='loss')
plt.plot(epochs, val_loss, label='val_loss')
plt.title('val_loss + loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, accuracy, label='accuracy')
plt.plot(epochs, val_accuracy, label='val_accuracy')
plt.title('val_accuracy + accuracy')
plt.legend()
plt.show()